In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

ModuleNotFoundError: No module named 'env'

In [ ]:
# Constant (to generate filename for csv)
filename = 'curriculum_logs_data.csv'

# Acquire data.
# ----------------------ACQUIRE FUNCTION---------------------------------
def acquire_curlogs():

    if os.path.isfile(filename):
        
        return pd.read_csv(filename)
        
    else: 

        query = '''
                SELECT *
                FROM cohorts
                LEFT JOIN logs ON cohorts.id = logs.user_id;
                '''

        url = get_connection('zillow')
                
        df = pd.read_sql(query, url)

        # save to csv
        df.to_csv(filename,index=False)

        return df

In [ ]:
curr_df = acquire_logs()
curr_df.info()

In [ ]:
curr_df.head()